In [1]:
import pymongo
import numpy as np
import pandas as pd
import datetime,time

In [2]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']
collection = db['tweets']

Connected successfully!!!


In [3]:
db.collection_names()

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['system.indexes', 'some-tweets', 'tweets', 'tweet_users']

In [4]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """
    
    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [5]:
df = read_mongo(db, "tweets")

In [7]:
import datetime,time
import numpy as numpy
import pandas as pd

cleaned = df[["id", "created_at","entities", "extended_entities","favorite_count","retweet_count","source","text", "truncated","user","quoted_status_id"]]
ids = [user["id"] for user in cleaned["user"]]
cleaned["id_num"] = ids
output = pd.DataFrame()
mean_RT_last10 = []
sd_RT = []
sd_FC = []
mean_FC_last10 = []
users = []

for _id in np.unique(ids):
    subset = cleaned[cleaned["id_num"]==_id]
    subset = subset.reset_index(drop=True)
    reindexed = subset["created_at"]
    reindexed = subset["created_at"].reset_index(drop=True)
    #convert tweet date into python date
    subset["created_at_pydate"] = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(reindexed[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(reindexed))]
    subset = subset.sort_values(by="created_at_pydate")
    last10_tweets = subset[-10:]
    #mean last 10 tweets
    mean_RT_last10.append(np.mean(last10_tweets["retweet_count"]))
    sd_RT.append(np.std(last10_tweets["retweet_count"]))

    mean_FC_last10.append(np.mean(last10_tweets["favorite_count"]))
    sd_FC.append(np.std(last10_tweets["favorite_count"]))
    #take json user
    j_user = subset[-1:]["user"].reset_index(drop=True)[0]
    users.append(j_user)
output["j_user"]=users
output["RT_l10"]=mean_RT_last10
output["sd_RT"]=s
output["FC_l10"]=mean_FC_last10

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
pd.DataFrame.to_csv(output, "C:/Users/Albert Prat/Documents/tweempact/datasets/author.csv")

In [10]:
np.std((2,0))

1.0